# Logistic regression on the adult census dataset

In [ ]:
import numpy as np
import pandas as pd  
from pandas.plotting import scatter_matrix
import seaborn as sns 
from seaborn import load_dataset
import statsmodels.api as sm
import matplotlib.pyplot as plt

#load imbalanced-learn library
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline

#Load sklearn libraries
from sklearn import metrics
from sklearn.metrics import get_scorer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import  StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics

#Load my custom transformers
from modules.preprocess import Drop_Columns
from modules.preprocess import InteractionsTransformer
from modules.preprocess import OrderFeatures
from modules.preprocess import Impute_Missing
from modules.preprocess import Predict_Missing
from modules.preprocess import drop_rows


columns = [
 'age',
 'workclass',
 'fnlwgt',
 'education',
 'educational num',
 'marital',
 'occupation',
 'relationship',
 'race',
 'gender',
 'capital gain',
 'capital loss',
 'hours per week',
 'country',
 'income'
 ]

#Load the dataset
train = pd.read_csv('data/adult.data', sep=",", names=columns)
test = pd.read_csv('data/adult.test', sep=",", names=columns)

#Drop missing
train = drop_rows(train, ['workclass', 'occupation', 'country'], drop_val =' ?')
test = drop_rows(test, ['workclass', 'occupation', 'country'], drop_val =' ?')

#preprocess
train['income'] = train['income'].str.strip(' .')
test['income'] = test['income'].str.strip(' .')

#Define and encode labels
train['income'].replace({'<=50K':0, '>50K':1}, inplace=True)
test['income'].replace({'<=50K':0, '>50K':1}, inplace=True)
y_train=train['income']
y_test=test['income']

#Define features and labels, the column education is already covered with the ordinal feature education.num
X_train = train.drop(['income'], axis='columns')
X_test = test.drop(['income'], axis='columns')

#Define model
#model = lgb.LGBMClassifier(random_state=0)
model = LogisticRegression(max_iter=500, random_state=0, class_weight={1:3, 0:1})

transformer = make_column_transformer(
    (StandardScaler(), ['age', 'educational num', 'capital gain', 'capital loss', 'hours per week']), #dont forget fnlwgt if in dataset!
    (OrdinalEncoder(), ['gender']),
    (OneHotEncoder(handle_unknown='ignore', sparse=False), ['workclass', 'marital', 'occupation', 'relationship']),
    remainder='passthrough')


#Compose pipeline
pipe = imbpipeline(steps = [
        ['orderfeatures', OrderFeatures()],
        ['drop_columns', Drop_Columns(['education', 'fnlwgt', 'country', 'race'])],
        #('impute_missing', Impute_Missing(missing_value=' ?', pred_columns=['workclass', 'occupation', 'country'], strategy = 'most_frequent' , label_missing=True)),
        #['predict_missing', Predict_Missing(pred_columns=['workclass', 'occupation', 'country'], print_cross_val_score=False, label_missing = True)],
        ['transformer', transformer],
        ['interactions', InteractionsTransformer(use_cache=False)],  
        #['smote', SMOTE(random_state=0)],
        ['model', model]
]
)

#Calculate scores
skf = StratifiedKFold(n_splits=5, random_state=0,  shuffle=True)

#Apply pipeline on data
pipe.fit(X_train, y_train)
y_test_predicted = pipe.predict(X_test)
y_train_predicted = pipe.predict(X_train)

print("Cross val train f1:", cross_val_score(pipe, X_train, y_train, cv=skf, scoring='f1').mean())
print("Overall train f1: ", get_scorer('f1')._score_func(y_train, y_train_predicted))
print("Overall test f1: ", get_scorer('f1')._score_func(y_test, y_test_predicted))

# Optimization of a logistic regression model
Here, I shortly discuss the approaches to improve the f1 score for a logistic model. To reproduce the results you just have to uncomment the transformers and functions in the cell above. For each step I describe which transformers were used.

Note: While optinizing the model, only the cross validation score on the training set was considered. The score on the test set was calculated at the very end of the project!

<table>
<thead>
  <tr>
    <th></th>
    <th>How?</th>
    <th>Results</th>
    <th>Conclusion</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>Base model</td>
    <td>Model<br>LogisticRegression(max_iter=500, random_state = 0) <br><br>Transformers<br>(StandardScaler(), ['age', ..]),<br>(OrdinalEncoder(), ['gender']),<br>(OneHotEncoder(handle_unknown='ignore', sparse=False), ['workclass', ..] <br>OrderFeatures()<br>Drop_Columns(['education', 'fnlwgt'])<br><br>
        <br>(have a look at the code above to see all parameters of the transformers)</td>
    <td><b>Cross val train f1: 0.66280</b><br>(Overall train f1:  0.66368)<br>(Overall test f1:  0.65735)</td>
    <td>I already dropped the column education in the base model because it is redundant.<br>Education_num contains the same information but is in a ordered numeric format.<br><br>Additionally, I dropped the column fnlwgt because it does not contain <br>relevant information for improving the f1 score. <br>However, one can think about weighing the observations with the fnlwgt values. <br>I don't do this here.<br>The goal of this ML project is just to train a model with high f1 score.</td>
  </tr>
  <tr>
    <td>Drop missing values</td>
    <td>I made two functions to do this:<br><br>drop_rows(train, ['workclass', 'occupation', 'country'], drop_val =' ?')<br>drop_rows(test,  ['workclass', 'occupation', 'country'], drop_val =' ?')</td>
    <td><b>Cross val train f1: 0.66673</b><br>(Overall train f1:  0.66836)<br>(Overall test f1:  0.66105)</td>
    <td>Better score when dropping rows with missing values.</td>
  </tr>
  <tr>
    <td>Impute missing</td>
    <td>I created a custom transformer to do this:<br><br>Impute_Missing(... strategy = 'most_frequent' , label_missing = False)<br><br>Of course the fitting is only done on the training data! We never wanna have data leakage!</td>
    <td><b>Cross val train f1: 0.66047</b><br>(Overall train f1:  0.66139)<br>(Overall test f1:  0.65579)<br></td>
    <td>Imputing with most frequent value is worse than dropping rows.</td>
  </tr>
  <tr>
    <td>Impute missing and <br>label imputed features</td>
    <td>I created a custom transformer to do this:<br><br>Impute_Missing(... strategy = 'most_frequent' , label_missing = True)<br><br>Of course the fitting is only done on the training data! We never wanna have data leakage!</td>
    <td><b>Cross val train f1: 0.66280</b><br>(Overall train f1:  0.663776)<br>(Overall test f1:  0.65735)</td>
    <td>Adding label to imputed values helps but score is still worse.</td>
  </tr>
  <tr>
    <td>Predict missing values<br>by ML</td>
    <td>I created a custom transformer to do this. Of course I fitted the model only to the training data!  <br>We never want data leakage!<br><br>Predict_Missing(pred_columns=['workclass', 'occupation', 'country'], .., label_missing = False)<br><br>Of course the fitting is only done on the training data! We never wanna have data leakage!</td>
    <td><b>Cross val train f1: 0.66063</b><br>(Overall train f1:  0.66241)<br>(Overall test f1:  0.65321)</td>
    <td>Predicting values with ML is worse than dropping rows.</td>
  </tr>
  <tr>
    <td>Predict missing values<br>by ML and label <br>predicted values</td>
    <td>I created a custom transformer to do this:<br><br>Predict_Missing(pred_columns=['workclass', 'occupation', 'country'], .., label_missing = True)<br><br>Of course the fitting is only done on the training data! We never wanna have data leakage!</td>
    <td><b>Cross val train f1: 0.66232</b><br>(Overall train f1:  0.66382)<br>(Overall test f1:  0.65763)</td>
    <td>Adding label to predicted values helps but score is still worse.<br><br>=&gt; we will drop rows with missing values for subsequent steps!</td>
  </tr>
  <tr>
    <td>Oversampling <br>with SMOTE</td>
    <td>SMOTE(random_state=0)<br><br>Of course the oversampling is only done on the training data and withing the CV steps! <br>We never wanna have data leakage!<br>For this purpose I used imbpipeline from <a href="https://imbalanced-learn.org/stable/">imbalanced-learn</a>.</td>
    <td><b>Cross val train f1: 0.68635</b><br>(Overall train f1:  0.68897)<br>(Overall test f1:  0.67750)</td>
    <td>Oversampling the minor class significantly improves the score</td>
  </tr>
  <tr>
    <td>Oversampling <br>with class weights</td>
    <td>LogisticRegression(.., class_weight={1:3, 0:1})<br>(see code above for all parameters)</td>
    <td><b>Cross val train f1: 0.68719</b><br>(Overall train f1:  0.68934)<br>(Overall test f1:  0.67836)</td>
    <td>Using the class weights model parameter <br>leads to slightly better results than SMOTE.<br>I will thus use class weights to handle the imbalanced targets for the next steps.</td>
  </tr>
  <tr>
    <td>Drop columns with low <br>permutation importance</td>
    <td>According to the calculated feature importance <br>race and country contain at least relevant information.<br>(Note: other columns were tested as well but the last two ones are documented here)<br><br>Drop_Columns(['education', 'fnlwgt', 'race', 'country'])</td>
    <td>Drop Country:<br><b>Cross val train f1: 0.68759</b><br>(Overall train f1:  0.68881)<br>(Overall test f1:  0.67784)<br><br>Drop Race:<br>Cross val train f1: 0.68647<br>(Overall train f1:  0.68851)<br>(Overall test f1:  0.67806)</td>
    <td>Race and Country were determined to have the lowest feature importance <br>(see <a href="Feature_Importance.ipynb">Feature_Importance.ipynb</a>).<br>When dropping country I observed a better f1 score. <br>When dropping Race the score remained about the same. <br><br>Thus, I only excluded the column country for the proceeding steps.<br>
    <br>Note:<br>
I also tested dropping other features with low permutation importance <br>but with other ones I could not increase the f1 score <br>(fnlwgt and education were already excluded in the base model!)
    </td>
  </tr>
  <tr>
    <td>Add interactions</td>
    <td>I made a custom transformer to do this. Of course I fitted the model only to the training data!  <br>We never want data leakage!<br><br>InteractionsTransformer()<br>Of course the fitting is only done on the training data! We never wanna have data leakage!</td>
    <td><b>Cross val train f1: 0.68879</b><br>(Overall train f1:  0.69217)<br>(Overall test f1:  0.679401)</td>
    <td>Adding interactions significantly improves the score<br>=&gt; we will keep it for further steps</td>
  </tr>
  <tr>
    <td>Hyperparameter tuning</td>
    <td>Because not all penalties work with all solvers I had to do several rounds of grid search.<br><br>Furthermore, its better to start with a larger parameter space.<br>In this way I can narrowing down to the optimal values.<br><br>Round 1<br>'model__C': [1, 2, 4, 8]<br>'model__penalty': ['l1', 'l2']<br>'model__solver': ['newton-cg', 'lbfgs']<br><br>Round 2<br>'model__C':[3, 5, 7],<br>'model__l1_ratio':[0, 0.2, 0.4],<br>'model__penalty': ['elasticnet'],<br>'model__solver': ['saga']<br><br>Round 3<br>'model__C':[4.1, 4.25, 4.3]<br>'model__penalty': ['l2'],<br>'model__solver': ['newton-cg']</td>
    <td><b>Cross val train f1: 0.68897</b><br>(Overall train f1:  0.69139)<br>(Overall test f1:  0.67885)</td>
    <td>By tuning the hyperparameters I could again slightly improve the score.<br><br><br></td>
  </tr>
</tbody>
</table>

# Hyperparameter tuning

In [2]:
#Define and encode labels
train['income'].replace({'<=50K':0, '>50K':1}, inplace=True)
test['income'].replace({'<=50K':0, '>50K':1}, inplace=True)
y_train=train['income']
y_test=test['income']

#Define features and labels, the column education is already covered with the ordinal feature education.num
X_train = train.drop(['income'], axis='columns')
X_test = test.drop(['income'], axis='columns')

#Define model
model = LogisticRegression(max_iter=500, random_state=0, class_weight={1:3, 0:1})

transformer = make_column_transformer(
    (StandardScaler(), ['age', 'educational num', 'capital gain', 'capital loss', 'hours per week']), #dont forget fnlwgt if in dataset!
    (OrdinalEncoder(), ['gender']),
    (OneHotEncoder(handle_unknown='ignore', sparse=False), ['workclass', 'marital', 'occupation', 'relationship']),
    remainder='passthrough')


#Compose pipeline
pipe = imbpipeline(steps = [
        ['orderfeatures', OrderFeatures()],
        ['drop_columns', Drop_Columns(['education', 'fnlwgt', 'country', 'race'])],
        #('impute_missing', Impute_Missing(missing_value=' ?', pred_columns=['workclass', 'occupation', 'country'], strategy = 'most_frequent' , label_missing=True)),
        #['predict_missing', Predict_Missing(pred_columns=['workclass', 'occupation', 'country'], print_cross_val_score=False, label_missing = True)],
        ['transformer', transformer],
        ['interactions', InteractionsTransformer(use_cache=True)],  
        #['smote', SMOTE(random_state=0)],
        ['model', model]
]
)

#Calculate scores
skf = StratifiedKFold(n_splits=5, random_state=0,  shuffle=True)

pipe = GridSearchCV(
        pipe,
            param_grid={
                'model__C':[4.1, 4.25, 4.3],    #[3, 5, 7]
                'model__penalty': ['l2'],       #['l1', 'l2'
                'model__solver': ['newton-cg']  #['newton-cg', 'lbfgs']
},
         cv=skf, refit=True, scoring = 'f1'
)

#Apply pipeline on data
pipe.fit(X_train, y_train)
y_test_predicted = pipe.predict(X_test)
y_train_predicted = pipe.predict(X_train)

print(pipe.best_score_)
print(pipe.best_params_)
print(pipe.best_estimator_)

0.6889776527280265
{'model__C': 4.25, 'model__penalty': 'l2', 'model__solver': 'newton-cg'}
Pipeline(steps=[('orderfeatures', OrderFeatures()),
                ('drop_columns',
                 Drop_Columns(columns=['education', 'fnlwgt', 'country',
                                       'race'])),
                ('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['age', 'educational num',
                                                   'capital gain',
                                                   'capital loss',
                                                   'hours per week']),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['ge

# Final model

In [36]:
#Define and encode labels
train['income'].replace({'<=50K':0, '>50K':1}, inplace=True)
test['income'].replace({'<=50K':0, '>50K':1}, inplace=True)
y_train=train['income']
y_test=test['income']

#Define features and labels, the column education is already covered with the ordinal feature education.num
X_train = train.drop(['income'], axis='columns')
X_test = test.drop(['income'], axis='columns')

#Define model
model = LogisticRegression(max_iter=500, random_state=0, class_weight={1:3, 0:1}, C=4.25, penalty='l2', solver='newton-cg')

transformer = make_column_transformer(
    (StandardScaler(), ['age', 'educational num', 'capital gain', 'capital loss', 'hours per week']), #dont forget fnlwgt if in dataset!
    (OrdinalEncoder(), ['gender']),
    (OneHotEncoder(handle_unknown='ignore', sparse=False), ['workclass', 'marital', 'occupation', 'relationship']),
    remainder='passthrough')


#Compose pipeline
pipe = imbpipeline(steps = [
        ['orderfeatures', OrderFeatures()],
        ['drop_columns', Drop_Columns(['education', 'fnlwgt', 'country', 'race'])],
        #('impute_missing', Impute_Missing(missing_value=' ?', pred_columns=['workclass', 'occupation', 'country'], strategy = 'most_frequent' , label_missing=True)),
        #['predict_missing', Predict_Missing(pred_columns=['workclass', 'occupation', 'country'], print_cross_val_score=False, label_missing = True)],
        ['transformer', transformer],
        ['interactions', InteractionsTransformer(use_cache=True)],  
        #['smote', SMOTE(random_state=0)],
        ['model', model]
]
)

#Calculate scores
skf = StratifiedKFold(n_splits=5, random_state=0,  shuffle=True)

#Apply pipeline on data
pipe.fit(X_train, y_train)
y_test_predicted = pipe.predict(X_test)
y_train_predicted = pipe.predict(X_train)

print("Cross val train f1:", cross_val_score(pipe, X_train, y_train, cv=skf, scoring='f1').mean())
print("Overall train f1: ", get_scorer('f1')._score_func(y_train, y_train_predicted))
print("Overall test f1: ", get_scorer('f1')._score_func(y_test, y_test_predicted))

Cross val train f1: 0.6889776527280265
Overall train f1:  0.6913952986844941
Overall test f1:  0.6788551529002068
